In [1]:
import gf as primpoly
import numpy as np

In [2]:
class GaloisField:
        
    def add(self,x, y):
        return x ^ y

    def sub(self,x, y):
        # in binary galois field, subtraction is just the same as addition (since we mod 2)
        return x ^ y 
    
    def mul(self,x, y):
        if x==0 or y==0:
            return 0
        return self.exp[(self.log[x] + self.log[y]) % (self.length-1)]

    def div(self,x,y):
        if y==0:
            raise ZeroDivisionError()
        if x==0:
            return 0
        return self.exp[(self.log[x] + (self.length-1) - self.log[y]) % (self.length-1)]

    def gf_pow(self, x, power):
        return self.exp[(self.log[x] * power) % (self.length-1)]
    
    def gf_inverse(self, x):
        return self.exp[(self.length-1) - self.log[x]]
    
    def conv(self,a,b):
        if (len(b) > len(a)):
            temp = a
            a = b
            b = temp

        a_pad = [*([0]*(len(b)-1)), *a, *([0]*(len(b)-1))]
        b_rev = list(reversed(b))

        res = [0]*(len(a_pad)-1)

        for i in range(0,len(a_pad)-len(b)+1):
            partial_sum = 0
            for j in range(0,len(b)):
                partial_sum = self.add(partial_sum,self.mul(a_pad[i+j],b[j]))
            res[i] = partial_sum

        return res
    
    def init_tables(self):
        
        '''Precompute the logarithm and anti-log tables for faster computation later, 
           using the provided primitive polynomial.'''
        # prim is the primitive (binary) polynomial. Since it's a polynomial in the binary sense,
        # it's only in fact a single galois field value between 0 and 255, and not a list of gf values.
        
        m = self.order
        prim = self.prim
        gf_length = self.length
        gf_exp = [0] * gf_length # anti-log (exponential) table
        gf_log = [0] * gf_length # log table
        # For each possible value in the galois field 2^8, 
        # we will pre-compute the logarithm and anti-logarithm (exponential) of this value
        x = 1
        for i in range(0, gf_length-1):
            gf_exp[i] = x # compute anti-log for this value and store it in a table
            gf_log[x] = i # compute log at the same time

            x <<= 1 # multiply by 2 (change 1 by another number y to multiply by a power of 2^y)
            if x >= gf_length:
                # substract the primary polynomial to the current value 
                # (instead of Galois Field length, so that we get a unique set made of coprime numbers), 
                # this is the core of the tables generation
                x ^= prim 
                
        return [gf_log, gf_exp]
    
    def det(self, matrix, mul = 1):
        '''Recursively computes the determinant of a square matrix in Galois Field'''
        
        width = len(matrix)
        if width == 1:
            return self.mul(mul, matrix[0][0])
        else:
            total = 0
            for i in range(width):
                m = []
                for j in range(1, width):
                    buff = []
                    for k in range(width):
                        if k != i:
                            buff.append(matrix[j][k])
                    m.append(buff)
                total = self.add(total, self.mul(mul, self.det(m, matrix[0][i])))
            return total
    
    def solveLinearSystem(self, matrix, array):
        '''Solves a linear system in Galois Field using Cramer's rule and 
           returns the answer to array ans[]''' 
        
        ans = []
        for i in range(0, len(matrix)):
            
            # We need to use deepcopy because temp = matrix returns a pointer to matrix
            # and all changes to temp also affect matrix
            temp = deepcopy(matrix)
            
            # We iteratively substitute each column of the matrix with the array
            # and the compute its determinant in the Galois Field
            for j in range(0, len(array)):
                temp[j][i] = array[j]

            ans.append(self.div(S.det(temp), self.det(matrix)))
        return ans
    
                
    def __init__(self, x, m, d):
        self.element = x
        self.order = m
        self.prim = d
        self.length = 2**m
        [self.log, self.exp] = self.init_tables()

In [3]:
t = 10 # maximum number of erroneous symbols that can be corrected
m = 4 # order of Galois Field
n = 2**m -1 # length of Galois Field
d = primpoly.find_prime_polys(2,m,single=True) 
a = GaloisField(2,m,d) # primitive element of Galois Field

In [4]:
# Compute the generator polynomial of the Reed Solomon code using convolution
# The first coefficient corresponds to x^(2t)
p = 0
g = GaloisField([1, a.gf_pow(a.element, p)], m, d)
for i in range(p+1,p+2*t):
    g.element = g.conv(g.element, [1, a.gf_pow(a.element, i)])
    
print("Generator polynomial of Reed Solomon code with m = {} and t = {}:\n{}".format(m,t,list(reversed(g.element))))

Generator polynomial of Reed Solomon code with m = 4 and t = 10:
[1, 12, 1, 4, 15, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 12, 1, 4, 15, 7]


In [5]:
field_el = GaloisField([],m,d)
for j in range(0,n):
    field_el.element.append(a.gf_pow(a.element,j))
    
print("The non zero elements of GF(2^{}) are:".format(m))
for j in range(0,n):
    print ("a^{} = {}".format(j,field_el.element[j]))

The non zero elements of GF(2^4) are:
a^0 = 1
a^1 = 2
a^2 = 4
a^3 = 8
a^4 = 3
a^5 = 6
a^6 = 12
a^7 = 11
a^8 = 5
a^9 = 10
a^10 = 7
a^11 = 14
a^12 = 15
a^13 = 13
a^14 = 9


In [6]:
# Find which elements a^i are included in the generator polynomial
power = np.zeros(len(g.element),dtype=int)
for i in range(0,len(g.element)):
    for j in range(0,n):
        if (g.element[i] == field_el.element[j]):
            power[i] = j
            
print("Exponents of a to which each coefficient of the generator polynomial corresponds to:\n{}".format(list(reversed(power))))

Exponents of a to which each coefficient of the generator polynomial corresponds to:
[0, 6, 0, 2, 12, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 2, 12, 10]
